<img src="https://www.hl7.org/fhir/assets/images/fhir-logo-www.png" style="float: left; width: 25%; margin-bottom: 0.5em;">

# Author: **Ricardo Paiva** ricardo.paiva@intersystems.com
- [Section 1: The FHIR HTTP API](#Section-1.-The-FHIR-REST-API)
- [Section 2: FHIR Search](#Section-2:-FHIR-Search)
- [Section 3: Search paramater types](#Section-3:-Search-parameter-types)
- [Section 4: Chaining and includes](#Section-4:-Chaining-and-includes)
- [Section 5: Putting it together](#Section-5:-Putting-it-together)

In [1]:
# save the base url of the FHIR server
base = 'http://localhost:9092/csp/healthshare/fhirserver/fhir/r4'

# setup imports
import os
from requests import get
from requests import post
from requests import put
from requests import delete
from requests import head
#from IPython.display import IFrame

## Section 1. The FHIR REST API

In [2]:
# This notebook focuses on the FHIR Search API, but first we use the "capabilities" interaction to learn about our target server.

# retrieve the server "CapabilityStatement" and print some important bits
response = get(base + '/metadata')
print('Response code: ' + str(response.status_code))
result = response.json()

print('Server: ' + result['software']['name'] + ' Version: ' + result['software']['version'])
#print('Security: ' + str(result['rest'][0]['security']))
resources = result['rest'][0]['resource']

supported_types = {r['type']: [i['code'] for i in r['interaction']] for r in resources}

print('Supported types: ')
for k,v in supported_types.items():
    print('  ' + k + ': ' + str(v))

Response code: 200
Server: InterSystems FHIR Server Version: 2020.2
Supported types: 
  Account: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  ActivityDefinition: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  AdverseEvent: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  AllergyIntolerance: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  Appointment: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  AppointmentResponse: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  AuditEvent: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  Basic: ['read', 'vread', 'create', 'update', 'patch', 'delete', 'history-instance', 'search-type']
  BodyStructure: ['read', 'vread', 'create', '

## Section 2: FHIR Search

In [3]:
# Now that we know our server supports the "search-type" interaction on all resource types, lets start working with the Patient endpoint.

# query for all Patient resources, then print the HTTP status code and the first 100 entries (default server config) of the response
response = get(base + '/Patient')
print('Response code: ' + str(response.status_code))
print('Response body: \n' + response.text)
print('Number of entries: ' + str(len(response.json().get('entry'))))

# technically you've now performed your first FHIR "search" (just with no parameters)

Response code: 200
Response body: 
{"resourceType":"Bundle","id":"783578ba-450b-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:41:52Z","total":613,"link":[{"relation":"first","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=1&queryId=782be0f2-450b-11eb-8af2-0242ac110003"},{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient"},{"relation":"next","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=2&queryId=782be0f2-450b-11eb-8af2-0242ac110003"},{"relation":"last","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=7&queryId=782be0f2-450b-11eb-8af2-0242ac110003"}],"entry":[{"fullUrl":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient/7","resource":{"resourceType":"Patient","id":"7","text":{"status":"generated","div":"<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthea</

In [5]:
# note that results are paged and the "link" field in the response Bundle contains links to the previous, current, and next page of results
for link in response.json().get('link'):
    if link.get('relation') == 'next':
        page2 = get(link.get('url'))        
print('Second page: \n' + page2.text)
print('Number of entries: ' + str(len(response.json().get('entry'))))

Second page: 
{"resourceType":"Bundle","id":"23001ae8-450c-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:46:39Z","total":613,"link":[{"relation":"first","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=1&queryId=782be0f2-450b-11eb-8af2-0242ac110003"},{"relation":"previous","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=1&queryId=782be0f2-450b-11eb-8af2-0242ac110003"},{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=2&queryId=782be0f2-450b-11eb-8af2-0242ac110003"},{"relation":"next","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=3&queryId=782be0f2-450b-11eb-8af2-0242ac110003"},{"relation":"last","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=7&queryId=782be0f2-450b-11eb-8af2-0242ac110003"}],"entry":[{"fullUrl":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient/77237","resource":{"resour

In [6]:
# we can control the number of resources on each page by passing the _count parameter
response = get(base + '/Patient?_count=1')
print('Single resource per page: \n' + response.text)
print('Number of entries: ' + str(len(response.json().get('entry'))))

Single resource per page: 
{"resourceType":"Bundle","id":"2add9e0c-450c-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:46:52Z","total":613,"link":[{"relation":"first","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=1&queryId=2ad73ea4-450c-11eb-8af2-0242ac110003"},{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?_count=1"},{"relation":"next","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=2&queryId=2ad73ea4-450c-11eb-8af2-0242ac110003"},{"relation":"last","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=613&queryId=2ad73ea4-450c-11eb-8af2-0242ac110003"}],"entry":[{"fullUrl":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient/7","resource":{"resourceType":"Patient","id":"7","text":{"status":"generated","div":"<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthe

In [7]:
# if you're only interested in the count, you can specify that via _count=0 (0 results per page);

print(get(base + '/Patient' + '?' + '_count=0').text)

{"resourceType":"Bundle","id":"2f5fbe24-450c-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:47:00Z","total":613,"link":[{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?_count=0"}],"entry":[]}


In [8]:
# for example, lets use the search parameter named "gender"
response = get(base + '/Patient' + '?' + 'gender=male')
print('Response code: ' + str(response.status_code))
print('Response body: \n' + response.text)

Response code: 200
Response body: 
{"resourceType":"Bundle","id":"33ffeef4-450c-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:47:07Z","total":335,"link":[{"relation":"first","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=1&queryId=33fb8094-450c-11eb-8af2-0242ac110003"},{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?gender=male"},{"relation":"next","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=2&queryId=33fb8094-450c-11eb-8af2-0242ac110003"},{"relation":"last","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?page=4&queryId=33fb8094-450c-11eb-8af2-0242ac110003"}],"entry":[{"fullUrl":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient/7","resource":{"resourceType":"Patient","id":"7","text":{"status":"generated","div":"<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea

In [8]:
# Let's explore the data

print('male:   \t' + str(get(base + '/Patient' + '?' + 'gender=male').json().get('total')))
print('female: \t' + str(get(base + '/Patient' + '?' + 'gender=female').json().get('total')))


male:   	335
female: 	278


## Section 3: Search parameter types

In [9]:
# search parameters have types

# telecom is considered a "token" search parameter

# Token search
# this parameter type is common for 'coded' values (Code, Coding, and CodeableConcept) and identifiers
# token values consist of a system and a code, although sometimes the system is implicit (think of gender)
# users can search on the system and code (system|code), the code alone (code), system-less codes (|code), or even the system alone (system|)
# we'll serach by phone number (patient.telecom.where(system='phone')

response = get(base + '/Patient' + '?' + 'telecom=555-374-9944')
print('Response code: ' + str(response.status_code))
print('Bundle Result:\n' + response.text)

# there are also Number, Date/DateTime, String, Reference, Quantity, URI, and Composite parameter types

Response code: 200
Bundle Result:
{"resourceType":"Bundle","id":"339e7864-4506-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:04:10Z","total":1,"link":[{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient?telecom=555-374-9944"}],"entry":[{"fullUrl":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Patient/7","resource":{"resourceType":"Patient","id":"7","text":{"status":"generated","div":"<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthea</a>.Version identifier: master-branch-latest\n .   Person seed: -803965613250436189  Population seed: 1595251654598</div>"},"extension":[{"url":"http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName","valueString":"Elisha578 Considine820"},{"url":"http://hl7.org/fhir/StructureDefinition/patient-birthPlace","valueAddress":{"city":"Rehoboth","state":"Massachusetts","country":"US"}},{"url":"http://synthetic

In [10]:
# String search
response = get(base + '/Patient' + '?' + 'family=Smith')
print('Smiths name evolution per Patient:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
        

Smiths name evolution per Patient:
265469: Murray856, Smith67
369055: Smith67


In [11]:
# String search performs a case-insensitive "begins-with" search by default!
# use the modifier ":exact" if you want exact matches (and improved performance)

response = get(base + '/Patient' + '?' + 'family:exact=Smith67')
print('Smiths name evolution per Patient:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))
print()

# string search also has a ":contains" modifier
response = get(base + '/Patient' + '?' + 'family:contains=will')
print('Will name evolution per Patient:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))


Smiths name evolution per Patient:
265469: Murray856, Smith67
369055: Smith67

Will name evolution per Patient:
264029: Williamson769, Goodwin327
377896: Willms744, Carroll471


In [12]:
# Date search
response = get(base + '/Patient' + '?' + 'birthdate=1984')
print('Born in 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

Born in 1984:
28637: 1984-06-20
79842: 1984-06-05
83586: 1984-03-23
180005: 1984-01-18
216075: 1984-11-09
233399: 1984-11-09
376168: 1984-03-03
391466: 1984-01-29
401581: 1984-02-29


In [13]:
# date searches support lt(<), le(<=), gt(>), ge(>=) "prefixes"
response = get(base + '/Patient' + '?' + 'birthdate=ge1984-01-01')
print('Born before 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))

response = get(base + '/Patient' + '?' + 'birthdate=gt1984-12-31')
print('\n' + 'Born after 1984:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(resource.get('birthDate'))


Born before 1984:
19306: 2003-10-17
20809: 2014-03-04
22563: 2012-12-27
22942: 2005-05-20
23324: 1986-03-15
24876: 1985-03-02
24992: 2002-12-30
25655: 1994-04-22
28637: 1984-06-20
30075: 2002-09-06
31786: 1997-11-28
33318: 2017-12-04
34560: 2017-09-08
35240: 1990-08-22
36607: 1985-12-16
40873: 2001-02-07
41097: 1989-08-19
43447: 2018-07-13
43651: 2010-06-12
46451: 1992-08-22
47467: 1996-11-15
47655: 1987-02-05
48870: 2013-07-18
49175: 1995-07-22
50709: 2013-10-08
51027: 2014-10-17
51945: 2020-02-04
53687: 2004-05-01
55550: 2014-09-12
55887: 2007-06-01
57556: 1992-12-25
58113: 1990-03-24
59803: 2010-09-27
68815: 2011-08-30
69202: 2007-06-12
70473: 1985-07-06
71898: 2005-09-18
72115: 1991-11-12
72604: 2002-01-22
73421: 2000-10-29
74366: 1985-08-29
75263: 2001-07-10
79842: 1984-06-05
81780: 2015-01-15
83586: 1984-03-23
84690: 2008-12-01
88429: 2009-06-26
93103: 1993-06-08
93722: 2009-11-24
94044: 1993-11-15
97361: 1993-09-10
100049: 1992-04-23
100235: 2011-11-27
100595: 1991-12-18
100747:

In [14]:
# Chained search
# Get all lab reports that have a subject which is a patient, whose name includes "Cassy".

response = get(base + '/DiagnosticReport?subject:Patient.name=Cassy')
print('Lab reports for patients named Cassy:   \n' + response.text)

Lab reports for patients named Cassy:   
{"resourceType":"Bundle","id":"8ad89a10-4506-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:06:36Z","total":96,"link":[{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/DiagnosticReport?subject:Patient.name=Cassy"}],"entry":[{"fullUrl":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/DiagnosticReport/21212","resource":{"resourceType":"DiagnosticReport","id":"21212","status":"final","category":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0074","code":"LAB","display":"Laboratory"}]}],"code":{"coding":[{"system":"http://loinc.org","code":"51990-0","display":"Basic Metabolic Panel"}],"text":"Basic Metabolic Panel"},"subject":{"reference":"Patient/21138"},"encounter":{"reference":"Encounter/21188"},"effectiveDateTime":"2011-12-11T01:50:03+00:00","issued":"2011-12-11T01:50:03.576+00:00","result":[{"reference":"Observation/21196","display":"Glucose"},{"reference":"Obser

In [15]:
# get all conditions that reference a specific patient
response = get(base + '/Condition' + '?' + 'subject=Patient/17524')
print('Conditions for patient 17524:')
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('code'))

# when the type of the reference is fixed to a single value, it can be omitted (Patient/x -> x)
response2 = get(base + '/Condition' + '?' + 'patient=17524')
print('\n' + 'Result entries match? ' + str(response.json().get('entry') == response2.json().get('entry')))

Conditions for patient 17524:
{'coding': [{'system': 'http://snomed.info/sct', 'code': '239873007', 'display': 'Osteoarthritis of knee'}], 'text': 'Osteoarthritis of knee'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '284551006', 'display': 'Laceration of foot'}], 'text': 'Laceration of foot'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '55822004', 'display': 'Hyperlipidemia'}], 'text': 'Hyperlipidemia'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '49727002', 'display': 'Cough (finding)'}], 'text': 'Cough (finding)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '386661006', 'display': 'Fever (finding)'}], 'text': 'Fever (finding)'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '840544004', 'display': 'Suspected COVID-19'}], 'text': 'Suspected COVID-19'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '840539006', 'display': 'COVID-19'}], 'text': 'COVID-19'}
{'coding': [{'system': 'http://snomed.info/sct', 'code': '2

## Section 4: Chaining and includes

In [16]:
# Chaining

# where reference parameters get really interesting is when you want to query one resource type based on a property of another resource to which its linked
# for example, here is a search for Type II Diabetes in female patients
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&' + 'patient:Patient.gender=female' + '&_count=1')
print('Type II Diabetes in female patients:   \n' + str(response.text))

Type II Diabetes in female patients:   
{"resourceType":"Bundle","id":"9ea7735e-4506-11eb-8af2-0242ac110003","type":"searchset","timestamp":"2020-12-23T10:07:09Z","total":21,"link":[{"relation":"first","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Condition?page=1&queryId=9ea57004-4506-11eb-8af2-0242ac110003"},{"relation":"self","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Condition?_count=1&code=http%3A//snomed.info/sct%7C44054006&patient:Patient.gender=female"},{"relation":"next","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Condition?page=2&queryId=9ea57004-4506-11eb-8af2-0242ac110003"},{"relation":"last","url":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Condition?page=21&queryId=9ea57004-4506-11eb-8af2-0242ac110003"}],"entry":[{"fullUrl":"http://localhost:9092/csp/healthshare/fhirserver/fhir/r4/Condition/25944","resource":{"resourceType":"Condition","id":"25944","clinicalStatus":{"coding":[{"system":"http://termin

In [21]:
# Includes

# its also possible to get a resource and its related resources back in a single query
response = get(base + '/Condition?code=http://snomed.info/sct|44054006' + '&' + '_include=Condition:patient' + '&_count=2')
print('Response contains both Conditions and Patients, but only the Conditions are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))


Response contains both Conditions and Patients, but only the Conditions are counted in the page size and total:
Total: 46
Condition: 26
Condition: 19818


In [22]:
# Reverse Includes

response = get(base + '/Patient?birthdate=ge2010-01-01' + '&' + '_revinclude=Condition:patient' + '&_count=2')
print('Response contains both Patients and Conditions, but only the Patients are counted in the page size and total:')
print('Total: ' + str(response.json().get('total')))
for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('resourceType'), end=': ')
    print(resource.get('id'))

Response contains both Patients and Conditions, but only the Patients are counted in the page size and total:
Total: 60
Patient: 20809
Patient: 22563


## Section 5: Putting it together

In [23]:
response = get(base + '/Condition' + '?' + 'code=http://snomed.info/sct|44054006' + '&_count=1')
print('Patients with Type II Diabetes:  ' + str(response.json().get('total')))

Patients with Type II Diabetes:  46


In [24]:
# SNOMED concepts for comorbidities of Type II Diabetes
#coronary heart disease (CHD), 53741008
#chronic kidney disease (CKD), 709044004
#atrial fibrillation, 49436004
#stroke, 230690007
#hypertension, 38341003
#heart failure, 84114007
#peripheral vascular disease (PVD), 400047006
#rheumatoid arthritis, 69896004
#Malignant neoplasm, primary (morphologic abnormality), 86049000
#Malignant neoplastic disease (disorder), 363346000
#osteoporosis, 64859006
#depression, 35489007
#asthma, 195967001
#chronic obstructive pulmonary disease (COPD), 13645005
#dementia, 52448006
#severe mental illness (SMI), 391193001
#epilepsy, 84757009
#hypothyroidism, 40930008
#learning disability, 1855002

print('CHD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '53741008').json().get('total')))
print('CKD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '709044004').json().get('total')))
print('AFib: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '49436004').json().get('total')))
print('stroke: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '230690007').json().get('total')))
print('hypertension: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '38341003').json().get('total')))
print('heart failure: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84114007').json().get('total')))
print('PVD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '400047006').json().get('total')))
print('arthritis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '69896004').json().get('total')))
print('cancer: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '86049000').json().get('total') + get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '363346000').json().get('total')))
print('osteoporosis: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '64859006').json().get('total')))
print('depression: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '35489007').json().get('total')))
print('asthma: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '195967001').json().get('total')))
print('COPD: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '13645005').json().get('total')))
print('dementia: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '52448006').json().get('total')))
print('SMI: \t\t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '391193001').json().get('total')))
print('epilepsy: \t\t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '84757009').json().get('total')))
print('hypothyroidism: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '40930008').json().get('total')))
print('learning disability: \t' + str(get(base + '/Condition?_summary=count&code=http://snomed.info/sct|' + '1855002').json().get('total')))

CHD: 			49
CKD: 			0
AFib: 			20
stroke: 		26
hypertension: 		0
heart failure: 		7
PVD: 			0
arthritis: 		2
cancer: 		0
osteoporosis: 		30
depression: 		0
asthma: 		1
COPD: 			0
dementia: 		0
SMI: 			0
epilepsy: 		18
hypothyroidism: 	0
learning disability: 	0


In [ ]:
# As a next step, why not find comorbidities on your population

---
FHIR® is the registered trademark of HL7 and is used with the permission of HL7. Use of the FHIR trademark does not constitute endorsement of this product by HL7.